# CA model for Membrane Permeability

Authors:
- H. Malash, 15179192
- S. Bhusal, studentnumber
- L. Pahladsingh, studentnumber
- T. Glansdorp, 12748587

In [ ]:
from model import CA_rules, CA_grid, CA_rules_only_water
import analysis
import numpy as np
import matplotlib.pyplot as plt
import random


### Counting of unbound water molecules

In [4]:
see_grid = CA_rules_only_water(CA_grid()).generate_simulation()
plt.imshow(see_grid)
plt.show()
total_height = 55
total_width = 55

unbound_water = analysis.count_unbound_water(see_grid, total_height, total_width)

NameError: name 'CA_rules_only_water' is not defined